In [ ]:
import pandas
import math
from matplotlib import pyplot as plt
import numpy
import tensorflow as tf
from tensorflow.python.framework import dtypes
import time


%matplotlib inline

DATA_LEN = 10000

def load_data(func):
    data = []
    for i in range (DATA_LEN):
        data.append(func(i))
    return pandas.DataFrame(data)

def split_data(data):
    return numpy.split(data, [int(.8*len(data))])

data_sin = load_data(lambda i: math.sin(i*0.01))
data_sinx = load_data(lambda i: math.sin(i*0.01)*(i*0.01))

train_data_sin, test_data_sin = split_data(data_sin)
train_data_sinx, test_data_sinx = split_data(data_sinx)

train_data_sin.plot()
test_data_sin.plot()

train_data_sinx.plot()
test_data_sinx.plot()

In [ ]:
sess = tf.InteractiveSession()
sess.as_default()
writer = tf.summary.FileWriter("/tmp/tensorflowlogs", sess.graph)

In [ ]:
class Model:
    def __init__(self, input_len, x, y, actual_activation, cost):
        self.input_len = input_len
        self.x = x
        self.y = y
        self.actual_activation = actual_activation
        self.cost = cost

def model_fully_connected(input_len):
    x = tf.placeholder("float", shape=(None, input_len))
    y = tf.placeholder("float", shape=(None, 1))
    W = tf.Variable(tf.zeros([input_len, 1]))
    b = tf.Variable(tf.zeros([1, 1]))
    actual_activation = tf.matmul(x, W) + b
    cost = tf.sqrt(tf.reduce_sum(tf.square(y - actual_activation)))
    
    return Model(input_len, x, y, actual_activation, cost)

def model_convolution(input_len, kernel_size, num_filters):
    x = tf.placeholder("float", shape=(None, input_len))
    xshaped = tf.reshape(x, [-1, input_len, 1])
    y = tf.placeholder("float", shape=(None, 1))
    conv1 = tf.layers.conv1d(
            inputs=xshaped,
            kernel_size=kernel_size,
            activation=tf.nn.relu,
            strides=1,
            filters=num_filters,
            padding="VALID")
    dimensions_for_fully_connected = (int)(num_filters*(input_len-(kernel_size))/2)
    max_pool = tf.reshape(
        tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='VALID'),
        [-1, dimensions_for_fully_connected]
    )
    W = tf.Variable(tf.zeros([dimensions_for_fully_connected, 1]))
    b = tf.Variable(tf.zeros([1, 1]))
    actual_activation = tf.matmul(max_pool, W) + b
    cost = tf.sqrt(tf.reduce_sum(tf.square(y - actual_activation)))
    
    return Model(input_len, x, y, actual_activation, cost)

def model_lstm(input_len, num_units):
    inputs = tf.placeholder(tf.float32, [None, input_len])
    inputs2 = tf.reshape(inputs, [-1, input_len, 1])
    correct_output = tf.placeholder(tf.float32, [None, 1])
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(num_units)])
    x_ = tf.unstack(inputs2, axis=1, num=input_len)
    output, layers = tf.contrib.rnn.static_rnn(stacked_lstm, x_, dtype=dtypes.float32)
    output = output[-1]
    prediction = tf.layers.dense(output, 1, activation=None)
    loss = tf.sqrt(tf.reduce_sum(tf.square(correct_output - prediction)))
    
    return Model(input_len, inputs, correct_output, prediction, loss)

def train(model, training_data, learning_rate, num_epochs=1):
    start = time.time()
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(model.cost)
    print("Training.")
    tf.global_variables_initializer().run()
    for j in range(num_epochs):
        for i in range(len(training_data.index)-(model.input_len + 1)): 
            batch_xs = training_data.iloc[i:i+model.input_len].T.values
            batch_ys = [training_data.iloc[i+model.input_len + 1].values]
            sess.run(optimizer, feed_dict={model.x: batch_xs, model.y: batch_ys})
        print("Epoch: ", j)
    end = time.time()
    print("Training done:", end - start, "s")
    
def train_full_batch(model, training_data, optimizer_name, learning_rate, num_epochs):
    start = time.time()
    if optimizer_name == "adagrad":
        optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(model.cost)
    else:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(model.cost) 
    print("Training.")
    tf.global_variables_initializer().run()
    batch_xs = []
    batch_ys = []
    for i in range(len(training_data.index)-(model.input_len + 1)):
        batch_xs.append(training_data.iloc[i:i+model.input_len].T.values[0])
        batch_ys.append(training_data.iloc[i+model.input_len+1].values)
    for j in range(num_epochs):
        sess.run(optimizer, feed_dict={model.x: batch_xs, model.y: batch_ys})
        if j % 100 == 0:
            print("Epoch: ", j)
    end = time.time()
    print("Training done:", end - start, "s")

def group_list(l, group_size):
    for i in range(0, len(l), group_size):
        yield l[i:i+group_size]
    
def train_batch(model, training_data, optimizer_name, learning_rate, batch_size, num_epochs):
    start = time.time()
    if optimizer_name == "adagrad":
        optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(model.cost)
    else:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(model.cost) 
    print("Training.")
    tf.global_variables_initializer().run()
    all_xs = []
    all_ys = []
    num_inputs = len(training_data.index)-(model.input_len + 1)
    for i in range(num_inputs):
        all_xs.append(training_data.iloc[i:i+model.input_len].T.values[0])
        all_ys.append(training_data.iloc[i+model.input_len+1].values)
    batch_xs = list(group_list(all_xs, batch_size))
    batch_ys = list(group_list(all_ys, batch_size))
    for j in range(num_epochs):
        for i in range(num_inputs // batch_size):
            sess.run(optimizer, feed_dict={model.x: batch_xs[i], model.y: batch_ys[i]})
        if j % 100 == 0:
            print("Epoch: ", j)
    end = time.time()
    print("Training done:", end - start, "s")
    
def replicate(model, input_data):
    test_input_x = []
    test_input_y = []
    for i in range(len(input_data.index)-(model.input_len + 1)):
        test_input_x.append(input_data.iloc[i:i+model.input_len].T.values[0])
        test_input_y.append(input_data.iloc[i+model.input_len+1].values)
    predictions_evaluated = model.actual_activation.eval(
        {model.x: test_input_x}
    )
    accuracy = tf.reduce_sum(tf.squared_difference (model.actual_activation, model.y))
    squared_diff = accuracy.eval({model.x: test_input_x, model.y: test_input_y})
    return pandas.DataFrame(numpy.concatenate((test_input_x[0], numpy.array(predictions_evaluated).flatten()))), squared_diff
    
def predict(model, input_data, steps):
    test_input_x = input_data.iloc[0:model.input_len].T.values.copy()
    predictions_evaluated = test_input_x[0].tolist()
    acc_sum = 0
    for i in range(steps):
        new_prediction = model.actual_activation.eval({model.x: test_input_x})[0]
        predictions_evaluated.extend (new_prediction)
        test_input_x[0] = numpy.concatenate((test_input_x[0][1:], numpy.array(new_prediction)))
        if (i < len(input_data)-(model.input_len+1)):
            acc_sum += (input_data.iloc[i+model.input_len][0] - new_prediction)**2
    return pandas.DataFrame(predictions_evaluated), acc_sum

def stats(model, train_data, test_data, steps):
    replicated_train, repl_error_train = replicate(model, train_data)
    replicated_test, repl_error_test = replicate(model, test_data)
    predicted_test, pred_error_test = predict(model, test_data, steps)
    print("Squared replication error (train loss):", repl_error_train)
    replicated_train.plot()
    print("Squared replication error (test loss):", repl_error_test)
    replicated_test.plot()
    print("Squared prediction error (test loss):", pred_error_test)
    predicted_test.plot()
    
def direct_compare(model, test_data):
    batch_xs = []
    batch_ys = []
    for i in range(len(test_data.index)-(model.input_len + 1)):
        batch_xs.append(test_data.iloc[i:i+model.input_len].T.values[0])
        batch_ys.append(test_data.iloc[i+model.input_len+1].values)

    predictions = model.actual_activation.eval(feed_dict={lstm_model.x: batch_xs})

    plot_predicted, = plt.plot(predictions, label='predicted')
    plot_test, = plt.plot(batch_ys, label='train')
    plt.legend(handles=[plot_predicted, plot_test])
    plt.show()

In [ ]:
fully_connected_model = model_fully_connected(200)
train(fully_connected_model, train_data_sin, 0.001, 20)
stats(fully_connected_model, train_data_sin, test_data_sin, 2000)

In [ ]:
fully_connected_model2 = model_fully_connected(200)
train_batch(fully_connected_model2, train_data_sinx, "default", 0.000001, 100, 200)
stats(fully_connected_model2, train_data_sinx, test_data_sinx, 2000)

In [ ]:
convolutional_model = model_convolution(200, 20, 10)
train(convolutional_model, train_data_sin, 0.001, 20)
stats(convolutional_model, train_data_sin, test_data_sin, 2000)

In [ ]:
convolutional_model2 = model_convolution(200, 10, 5)
train_batch(convolutional_model2, train_data_sinx, "default", 0.00001, 100, 300)
stats(convolutional_model2, train_data_sinx, test_data_sinx, 2000)

In [ ]:
lstm_model = model_lstm(100, 128) # 5,3 / 10,10
train_batch(lstm_model, train_data_sin, "adagrad", 0.01, 100, 5) # 0.1 1000 /
stats(lstm_model, train_data_sin, test_data_sin, 2000)